### Load libs

In [1]:
import pandas as pd
import json
import numpy as np
import random
from pandarallel import pandarallel
import re
import time
import multiprocessing
from multiprocessing import Pool
from joblib import Parallel, delayed
import sys

pandarallel.initialize()


INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
n = 750
pd.set_option('display.max_columns', n)
pd.set_option('display.max_rows', n)
pd.set_option('display.max_colwidth', -1)


/anaconda3/envs/kn/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


### Load data

In [3]:
term = 'Santander Drive Auto Receivables Trust 2020-4 Data Tape'
finder = re.compile('\d{4,}\W\d{1,}')
add_id = re.findall(finder, term)[0]
add_id


'2020-4'

In [4]:
# load abs
folder = 'data/transaction/'
file = '{}.csv'.format(term)
path = folder + file
data = pd.read_csv(path)
init_shape = data.shape[0]
data.shape


(366189, 73)

In [5]:
# load fields
f_folder = 'data/json/fields/'
f_file = 'fields.json'
f_path = f_folder + f_file
with open(f_path) as f:
    fields = json.load(f)


In [6]:
# load mapper
m_folder = 'data/dictionary/mapper/'
m_file = 'mapper.json'
m_path = m_folder + m_file
with open(m_path) as f:
    mapper = json.load(f)
    

### Setting fields

In [7]:
init_id = fields['init_id'][0]
date_cols = fields['dates']
replacer_cols = fields['replace_dash']
clean_cols = fields['clean']
m_cols = fields['map']
event_cols = fields['event']
loc_cols = fields['all_loc']
numeric_cols = fields['numeric']
all_vals_cols = fields['all_vals']
min_max_cols = fields['min_max']
west = fields['regions']['west']
south_west = fields['regions']['south_west']
south_east = fields['regions']['south_east']
mid_west = fields['regions']['mid_west']
north_east = fields['regions']['north_east']


### ID and dates

In [8]:
def reorder_date(init):
    
    """
    Reorder date
    """
    
    init = str(init)
    if init != '-':
        if '/' not in init:
            y = init[6:10]
            m = init[0:2]
            d = init[3:5]
            date = y + '-' + m + '-' + d
        elif '/' in init:
            y = init[3:7]
            m = init[0:2]
            date = y + '-' + m
    else:
        date = ''
    
    return date


In [9]:
data['id'] = data[init_id].str.replace('=', '').str.replace('"', '').str.strip() + '-' + add_id


In [10]:
for col in date_cols:
    print(col)
    values = data[col].values
    dates = [reorder_date(v) for v in values]
    data['{}R'.format(col)] = dates
    

reportingPeriodBeginningDate
DemandResolutionDate
reportingPeriodEndingDate
loanMaturityDate
originationDate
interestPaidThroughDate
originalFirstPaymentDate
mostRecentServicingTransferReceivedDate
zeroBalanceEffectiveDate


In [11]:
data = data.sort_values(by=['reportingPeriodBeginningDateR'], ascending=False)
data = data.drop_duplicates(subset = ['id', 'reportingPeriodBeginningDateR'], keep = 'first')
sec_shape = data.shape[0]
data.shape

(302642, 83)

In [12]:
if init_shape == sec_shape:
    dup_rows = False
    print('Matching shapes')
else:
    dup_rows = True 
    print('Duplicate rows')

Duplicate rows


In [13]:
# s_col = 'loanMaturityDate'
# t_col = '{}R'.format(s_col)
# data[[s_col, t_col]]


### Replacing values

In [14]:
data[replacer_cols] = data[replacer_cols].replace('-', np.nan)


In [15]:
# clean cols
for col in clean_cols:
    data[col] = data[col].astype(str)
    data[col] = data[col].str.strip()
    data[col] = data[col].astype(float)
    

### Replacing values

In [16]:
def replace_val(init, column):
    
    """
    Replace numeric values
    """
    
    init = str(init).strip().replace(';', '')
    if init in ['0', '1', '2', '3', '4', '5', '98', '99']:
        mapped = mapper[column][init]
        return mapped
    else:
        if init[0] in ['0', '1', '2', '3', '4', '5']:
            use = init[0]
        elif init == '-':
            use_keys = list(mapper[column].keys())
            if '98' in use_keys:
                use = '98'
            elif '99' in use_keys:
                use = '99'
        else:
            use = init
        mapped = mapper[column][use]
        
    return mapped
    

In [17]:
for col in m_cols:
    print(col)
    values = data[col].values
    ret_vals = [replace_val(v, col) for v in values]
    data['{}M'.format(col)] = ret_vals
    

obligorIncomeVerificationLevelCode
interestCalculationTypeCode
zeroBalanceCode
modificationTypeCode
assetSubjectDemandStatusCode
repurchaseReplacementReasonCode
vehicleValueSourceCode
obligorEmploymentVerificationCode
vehicleNewUsedCode
vehicleTypeCode
paymentTypeCode
servicingAdvanceMethodCode
originalInterestRateTypeCode
subvented


In [18]:
# s_col = 'subvented'
# t_col = '{}M'.format(s_col)
# data[[s_col, t_col]]


### Account status

In [19]:
def acct_status(row, b_col, e_col, zero_col, thresh):
    
    """
    Create karus account status
    """
    
    b = float(row[b_col])
    e = float(row[e_col])
    z = str(row[zero_col])
    
    if z in ['Charged-off', 'Repurchased or Replaced']:
        res = z
        return res
    if b < thresh and e < thresh:
        res = 'Prepaid or Matured'
        return res
    if z in ['Unavailable', 'Prepaid or Matured']:
        res = z
        return res
    

In [20]:
b_col = 'reportingPeriodBeginningLoanBalanceAmount'
e_col = 'nextReportingPeriodPaymentAmountDue'
z_col = 'zeroBalanceCodeM'
thresh = 50


In [21]:
data['accountStatus'] = data.parallel_apply(acct_status, args = (b_col, e_col, z_col, thresh, ), axis = 1)


In [22]:
data['accountStatus'].value_counts()


Unavailable                300268
Prepaid or Matured         2249  
Repurchased or Replaced    88    
Charged-off                37    
Name: accountStatus, dtype: int64

### Numeric conversion

In [23]:
# force convert cols to numeric
for col in numeric_cols:
    print(col)
    data[col] = pd.to_numeric(data[col], errors='coerce')
    

currentDelinquencyStatus
paymentToIncomePercentage
paymentExtendedNumber
otherAssessedUncollectedServicerFeeAmount
totalActualAmountPaid
actualInterestCollectedAmount
remainingTermToMaturityNumber
reportingPeriodScheduledPaymentAmount
vehicleValueAmount
reportingPeriodInterestRatePercentage
scheduledInterestAmount
actualPrincipalCollectedAmount
actualOtherCollectedAmount
reportingPeriodActualEndBalanceAmount
servicingFlatFeeAmount
chargedoffPrincipalAmount
servicingFeePercentage
otherPrincipalAdjustmentAmount
obligorCreditScore
originalInterestRatePercentage
nextReportingPeriodPaymentAmountDue
gracePeriodNumber
originalLoanTerm
repurchaseAmount
recoveredAmount
servicerAdvancedAmount
originalLoanAmount
repossessedProceedsAmount
nextInterestRatePercentage
scheduledPrincipalAmount
reportingPeriodBeginningLoanBalanceAmount


### Application

In [24]:
id_col = 'id'
status_col = 'accountStatus'
values = ['Charged-off', 'Prepaid or Matured', 'Repurchased or Replaced']


In [25]:
all_ids = list(data[id_col].unique())
#all_ids = all_ids[:1000]
print_vals = list(range(0, len(all_ids), 100))
len(all_ids)


76183

In [26]:
# break ids into list chunks
num = 1000
id_lists = [all_ids[i:i + num] for i in range(0, len(all_ids), num)]  
#id_lists = [id_lists[-1]]


In [27]:
len(id_lists)

77

In [28]:
def convert_static(df, exception_status):
    
    """
    Create static df
    """
    
    #df = init.reset_values(drop = True)
    df.reset_index(drop = True, inplace = True)
    df = df.sort_values('reportingPeriodBeginningDateR', ascending = False)
    df['indexAccount'] = df.index
    _id = df[id_col].iloc[0]

    # dict
    account_dict = {}
    account_dict['exceptionStatus'] = exception_status
    account_dict[id_col] = _id
    account_dict['records'] = len(df)

    # current status of loan
    for col in loc_cols:
        account_dict['{}LocCurrent'.format(col)] = df[col].iloc[0]
    for col in min_max_cols:
        account_dict['{}MaxCurrent'.format(col)] = df[col].max()
        account_dict['{}MinCurrent'.format(col)] = df[col].min()
    for col in all_vals_cols:
        vals = list(df[col].unique())
        use_vals = ' | '.join(str(val) for val in vals)
        account_dict['{}ValsCurrent'.format(col)] = use_vals
    for col in numeric_cols:
        _sum = df[col].sum()
        account_dict['{}SumCurrent'.format(col)] = _sum
        vec = list(df[col])
        vec = [v for v in vec if str(v) != 'nan']
        if len(vec) > 0:
            _len = len(vec)
            weights = sorted([1 + i for i in list(range(_len))], reverse=True)
            wa = np.average(vec, weights=weights)
            account_dict['{}WeightedCurrent'.format(col)] = wa
        else:
            account_dict['{}WeightedCurrent'.format(col)] = 0

    # event information
    init_vals = list(df[status_col].unique())
    inter = list(set(values).intersection(init_vals))
    if len(inter) > 0:
        account_dict['eventOccurred'] = 1
        n = df[status_col].where(df[status_col].isin(values)).last_valid_index()
        account_dict['eventIndex'] = n
        n_bool = True
        single = df.loc[[n]]
        for col in event_cols:
            account_dict['{}Event'.format(col)] = single[col].iloc[0]

        # prior to event
        init = n+1
        sub = df[init:len(df)]
        sub.reset_index(drop = True, inplace = True)
        account_dict['priorHistory'] = len(sub)
        sub_bool = True
        if len(sub) > 0:
            for col in loc_cols:
                account_dict['{}LocPrior'.format(col)] = sub[col].iloc[0]
            for col in min_max_cols:
                account_dict['{}MinPrior'.format(col)] = sub[col].min()
                account_dict['{}MaxPrior'.format(col)] = sub[col].max()
            for col in all_vals_cols:
                vals = list(sub[col].unique())
                use_vals = ' | '.join(str(val) for val in vals)
                account_dict['{}ValsPrior'.format(col)] = use_vals
            for col in numeric_cols:
                account_dict['{}SumPrior'.format(col)] = sub[col].sum()
                vec = list(sub[col])
                vec = [v for v in vec if str(v) != 'nan']
                if len(vec) > 0:
                    _len = len(vec)
                    weights = sorted([1 + i for i in list(range(_len))], reverse=True)
                    wa = np.average(vec, weights=weights)
                    account_dict['{}WeightedPrior'.format(col)] = wa
                else:
                    account_dict['{}WeightedPrior'.format(col)] = 0

            # random
            len_sub = len(sub)            
            s = random.randint(0, len_sub)
            if s == len_sub:
                s = s -1
            r_sub = sub[s:len_sub].reset_index(drop = True)
            account_dict['randomIndex'] = s
            for col in loc_cols:
                account_dict['{}LocRandom'.format(col)] = r_sub[col].iloc[0]
            for col in min_max_cols:
                account_dict['{}MinRandom'.format(col)] = r_sub[col].min()
                account_dict['{}MaxRandom'.format(col)] = r_sub[col].max()
            for col in all_vals_cols:
                vals = list(r_sub[col].unique())
                use_vals = ' | '.join(str(val) for val in vals)
                account_dict['{}ValsRandom'.format(col)] = use_vals
            for col in numeric_cols:
                account_dict['{}SumRandom'.format(col)] = r_sub[col].sum()
                vec = list(r_sub[col])
                vec = [v for v in vec if str(v) != 'nan']
                if len(vec) > 0:
                    _len = len(vec)
                    weights = sorted([1 + i for i in list(range(_len))], reverse=True)
                    wa = np.average(vec, weights=weights)
                    account_dict['{}WeightedRandom'.format(col)] = wa
                else:
                    account_dict['{}WeightedRandom'.format(col)] = 0

        # if event is first row of sub       
        else:
            for col in loc_cols:
                account_dict['{}LocPrior'.format(col)] = df[col].iloc[0]
            for col in min_max_cols:
                account_dict['{}MinPrior'.format(col)] = df[col].min()
                account_dict['{}MaxPrior'.format(col)] = df[col].max()
            for col in all_vals_cols:
                vals = list(df[col].unique())
                use_vals = ' | '.join(str(val) for val in vals)
                account_dict['{}ValsPrior'.format(col)] = use_vals
            for col in numeric_cols:
                account_dict['{}SumPrior'.format(col)] = df[col].sum()
                account_dict['{}WeightedPrior'.format(col)] = df[col].iloc[0]

    # if no event        
    else:
        account_dict['eventOccurred'] = 0
        account_dict['priorHistory'] = len(df)
        sub_bool = False
        n_bool = False
        for col in event_cols:
            account_dict['{}Event'.format(col)] = np.nan
        for col in loc_cols:
            account_dict['{}LocPrior'.format(col)] = np.nan
        for col in min_max_cols:
            account_dict['{}MinPrior'.format(col)] = np.nan
            account_dict['{}MaxPrior'.format(col)] = np.nan
        for col in all_vals_cols:
            account_dict['{}ValsPrior'.format(col)] = np.nan
        for col in numeric_cols:
            account_dict['{}SumPrior'.format(col)] = np.nan
            account_dict['{}WeightedPrior'.format(col)] = np.nan

        # random set to nan
        account_dict['randomIndex'] = np.nan
        for col in loc_cols:
            account_dict['{}LocRandom'.format(col)] = np.nan
        for col in min_max_cols:
            account_dict['{}MinRandom'.format(col)] = np.nan
            account_dict['{}MaxRandom'.format(col)] = np.nan
        for col in all_vals_cols:
            account_dict['{}ValsRandom'.format(col)] = np.nan
        for col in numeric_cols:
            account_dict['{}SumRandom'.format(col)] = np.nan
            account_dict['{}WeightedRandom'.format(col)] = np.nan

    return account_dict


In [29]:
log = {}
log['securitization'] = term
master_list = []
broke = 0
cautions = 0
status = 'good'
s1 = time.time()
ids_count = 0
for ids in id_lists:
    
    # get update
    ids_count = ids_count + len(ids)
    percent = ids_count / len(all_ids)
    print(ids_count, percent)
    
    # create sub
    sub_df = data[data[id_col].isin(ids)]
    sub_df['indexTransaction'] = sub_df.index
    sub_df = sub_df.reset_index(drop = True)
    splits = list(sub_df.groupby(id_col)) 
    l = [splits[n_][1] for n_ in list(range(len(splits)))]
    a = np.array(l)
    
    # get results
    s2 = time.time()
    try:
        exception_status = False
        results = [convert_static(d, exception_status) for d in a]
        for r in results:
            master_list.append(r)
    except:
        try:
            exception_status = True 
            cautions = cautions + 1
            print('HITTING EXCEPTION')
            for l2 in l:
                res = convert_static(l2, exception_status)
                master_list.append(res)
        except:
            status = 'bad'
            broke = broke + 1
            if broke > 0:
                sys.exit('Too many errors...')
    
    e1 = time.time()
    e2 = time.time()
    log['{}'.format(ids_count)] = (e2 - s2)
    
    print(e2 - s2)
    print(e1 - s1)
    print('-----------------------------')
    

1000 0.01312628801701167


/anaconda3/envs/kn/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


HITTING EXCEPTION
15.818984985351562
16.678770780563354
-----------------------------
2000 0.02625257603402334
HITTING EXCEPTION
16.023860931396484
33.25091910362244
-----------------------------
3000 0.03937886405103501
HITTING EXCEPTION
15.082980155944824
48.908987045288086
-----------------------------
4000 0.05250515206804668
HITTING EXCEPTION
15.36239504814148
64.7528748512268
-----------------------------
5000 0.06563144008505835
HITTING EXCEPTION
16.711899995803833
82.0361099243164
-----------------------------
6000 0.07875772810207002
HITTING EXCEPTION
17.747890949249268
100.35337281227112
-----------------------------
7000 0.09188401611908169
HITTING EXCEPTION
17.532722234725952
118.43884205818176
-----------------------------
8000 0.10501030413609336
HITTING EXCEPTION
18.03433918952942
137.11513805389404
-----------------------------
9000 0.11813659215310503
HITTING EXCEPTION
14.332783222198486
152.05483984947205
-----------------------------
10000 0.1312628801701167
HITTING 

/anaconda3/envs/kn/lib/python3.7/site-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


14.773522853851318
269.79128789901733
-----------------------------
18000 0.23627318430621005
HITTING EXCEPTION
14.483197927474976
284.8227639198303
-----------------------------
19000 0.2493994723232217
14.104943990707397
299.39719700813293
-----------------------------
20000 0.2625257603402334
HITTING EXCEPTION
14.910669088363647
314.77772998809814
-----------------------------
21000 0.27565204835724505
HITTING EXCEPTION
15.32523488998413
330.6056020259857
-----------------------------
22000 0.2887783363742567
14.89021897315979
345.9832260608673
-----------------------------
23000 0.3019046243912684
HITTING EXCEPTION
13.282132148742676
359.69128012657166
-----------------------------
24000 0.3150309124082801
HITTING EXCEPTION
13.919404029846191
374.0814571380615
-----------------------------
25000 0.32815720042529173
HITTING EXCEPTION
13.58312201499939
388.20246291160583
-----------------------------
26000 0.3412834884423034
HITTING EXCEPTION
13.889235973358154
402.57178378105164
---

In [30]:
log['total_time'] = e1 - s1
log['run_status'] = status
log['cautions'] = cautions
log['duplicate_rows'] = dup_rows


In [31]:
master = pd.DataFrame(master_list)
log['loans'] = len(master)


In [32]:
cs = master['obligorCreditScoreLocCurrent'].mean()
if cs <= 600:
    rating = 'sub_prime'
elif cs > 600 and cs <= 700:
    rating = 'near_prime'
elif cs > 720:
    rating = 'prime'
else:
    rating = 'other'
log['average_credit'] = cs
log['rating'] = rating


In [33]:
len(all_ids) == len(master_list)

True

In [34]:
master['accountStatusEvent'].value_counts(dropna = False)

NaN                        73836
Prepaid or Matured         2232 
Repurchased or Replaced    80   
Charged-off                35   
Name: accountStatusEvent, dtype: int64

### Adding final fields

In [35]:
master['securitization'] = term


In [36]:
def get_target(row):
    
    """
    Set target var
    """
    
    init = str(row['accountStatusEvent'])
    remaining = row['remainingTermToMaturityNumberMinPrior']
    #remaining = row['remainingTermToMaturityNumberLocCurrent']
    #remaining = row['remainingTermToMaturityNumberLocPrior']
    
    if init == 'Charged-off':
        res = 'Charged-off'
        return res
    elif init == 'Prepaid or Matured' and remaining > 1:
        res = 'Prepaid'
        return res
    elif init == 'Prepaid or Matured' and remaining < 2:
        res = 'Closed'
        return res
    elif init == 'nan':
        res = 'Active or other'
        return res
    else:
        res = 'Active or other'
        return res
    

In [37]:
master['target'] = master.apply(get_target, axis = 1)

In [38]:
master['target'].value_counts(dropna = False)

Active or other    73916
Prepaid            2232 
Charged-off        35   
Name: target, dtype: int64

In [39]:
def finding_regions(state):
    
    """
    Get region
    """
    
    if state in west:
        return 'West'
    elif state in south_west:
        return 'SouthWest'
    elif state in south_east:
        return 'SouthEast'
    elif state in mid_west:
        return 'MidWest'
    elif state in north_east:
        return 'NorthEast'
    else:
        return 'Unknown'
    

In [40]:
master['region'] = np.nan
    
master['region'] = master['obligorGeographicLocationLocCurrent'].apply(finding_regions)

In [41]:
def get_or_year(init):
    
    """
    Get origination year
    """
    
    year = init[0:4]
    
    return year


In [42]:
year_col = 'originationDateRLocCurrent'

year_vals = master[year_col].values
years = [get_or_year(y) for y in year_vals]
master['originationYear'] = years


In [43]:
master['originationDate'] = pd.to_datetime(master['originationDateRLocCurrent'])

In [44]:
master.shape

(76183, 713)

In [45]:
#master[master['exceptionStatus'].isin([True])]

### Adding outcome for transaction file

In [46]:
m_col = 'id'
m_type = 'left'
merged = pd.merge(data, master[[m_col, 'target']], on = m_col, how = m_type)
ft_shape = merged.shape[0]

In [47]:
ft_shape

302642

In [48]:
final_bool = init_shape == sec_shape == ft_shape
log['lengths_match'] = final_bool


In [49]:
final_bool

False

### Vals cols

In [50]:
vals_cols = [col for col in list(master.columns) if 'vals' in col.lower()]

In [51]:
def fix_vals(row, column):

    """
    Fix column values
    """

    init = str(row[column])

    ret_val = 'str: ' + init

    return ret_val

In [52]:
for col in vals_cols:
    print(col)
    master[col] = master[col].astype(str)
    master[col] = 'str: ' + master[col]

obligorIncomeVerificationLevelCodeValsCurrent
interestCalculationTypeCodeValsCurrent
zeroBalanceCodeValsCurrent
subventedValsCurrent
modificationTypeCodeValsCurrent
assetSubjectDemandStatusCodeValsCurrent
repurchaseReplacementReasonCodeValsCurrent
vehicleValueSourceCodeValsCurrent
accountStatusValsCurrent
obligorEmploymentVerificationCodeValsCurrent
originalInterestRateTypeCodeValsCurrent
vehicleNewUsedCodeValsCurrent
vehicleTypeCodeValsCurrent
paymentTypeCodeValsCurrent
indexAccountValsCurrent
servicingAdvanceMethodCodeValsCurrent
indexTransactionValsCurrent
zeroBalanceCodeMValsCurrent
obligorIncomeVerificationLevelCodeValsPrior
interestCalculationTypeCodeValsPrior
zeroBalanceCodeValsPrior
subventedValsPrior
modificationTypeCodeValsPrior
assetSubjectDemandStatusCodeValsPrior
repurchaseReplacementReasonCodeValsPrior
vehicleValueSourceCodeValsPrior
accountStatusValsPrior
obligorEmploymentVerificationCodeValsPrior
originalInterestRateTypeCodeValsPrior
vehicleNewUsedCodeValsPrior
vehicleT

### Export

In [53]:
e_folder = 'data/static/'
e_file = '{} static.csv'.format(term)
e_path = e_folder + e_file
print(e_path)
print(master.shape)
master.to_csv(e_path, index = False)


data/static/Santander Drive Auto Receivables Trust 2020-4 Data Tape static.csv
(76183, 713)


In [54]:
t_folder = 'data/transaction/prepared/'
t_file = '{} transaction.csv'.format(term)
t_path = t_folder + t_file
print(t_path)
print(merged.shape)
merged.to_csv(t_path)


data/transaction/prepared/Santander Drive Auto Receivables Trust 2020-4 Data Tape transaction.csv
(302642, 99)


In [55]:
# export log
j_folder = 'data/static/log/'
j_file = '{} log.json'.format(term)
j_path = j_folder + j_file
with open(j_path, 'w') as outfile:  
    json.dump(log, outfile, indent = 4, separators = (',', ': '), sort_keys = False)


In [56]:
print('continue...')


continue...


### End